

<center>

# Trabajo Final: Detección y Segmentación de TN en Imágenes de Ultrasonido

## Procesamiento Avanzado de Imágenes en Biomedicina y Biología

### Segundo Cuatrimestre 2024

<table>
      <tr>
        <th>Alumnos</th>
        <th>Legajos</th>
      </tr>
      <tr>
        <td>
          Bajlec, Ivo<br>
          Grau, Gonzalo Andrés<br>
          Neira, Lucas Matias
        </td>
        <td>
          62175<br>
          62259<br>
          61750<br>
        </td>
      </tr>
    </table>

**Docente**: Roberto Sebastián Tomás

**Fecha de entrega**: 09/12/2024

</center>


# Objetivo del trabajo
El objetivo del presente trabajo es desarrollar algoritmos para identificar y segmentar la translucencia nucal a partir de imágenes de ultrasonido.
Para ello, se aplicarán técnicas avanzadas de reducción de ruido y preprocesamiento, para luego entrenar redes neuronales convolucionales.
Los resultados de este estudio podrían contribuir significativamente al campo de la obstetricia, al proporcionar una herramienta de asistencia a la toma de deciiones, obteniéndose así una estimación de la medida de la TN en forma independiente del operador.

# Introducción
En 1985, se descubrió que existe una correlación entre el grosor del pliegue nucal en fetos del segundo trimestre y las aparación de alteraciones cromosómicas. Luego, se descubrió que este fenómeno también es identificable en el primer trimestre, y se introdujo el término “translucencia nucal” (TN) para describir la región sonoluscente en la parte posterior de la nuca fetal. Estudios realizados en gestantes de alto riesgo confirmaron esta relación. [4]

Para cuantificar esta relación, se desarrolló un modelo matemático que combinaba el riesgo asociado a la TN con la edad materna y los marcadores bioquímicos: el método de cribado combinado. Siguiente el protocolo adecuado, descito por la Fetal Medicine Foundation [3], se toma la medición del máximo ancho de un área anecogénica en la parte dorsal del embrión, a partir de una imagen ampliada que incluya solamente la cabeza y el toráx en una visión sagital. Una de las principales fuentes de confusión a la hora de tomar estas medidas es identificar como área de la TN al espacio entre la membrana amniótica y la piel fetal, el cual tiene una mofología similar a la del área de interés. 

Un algoritmo especializado para la detección del área de la TN ayudaría al profesional médico a detectar adecuadamente la región de interés (ROI), mientras que una herramienta de segmentación permitiría cuantifica el ancho de la TN en forma objetiva e independiente del operador.
En el siguiente trabajo, se desarrollarán herramientas basadas en redes neuronales convolucionales (CNN) para resolver ambos desafíos.
Para la detección, se aplicará un reajuste de pesos sobre una arquitectura VGG16, entrenándola para obtener bounding boxes que encierren a la ROI mediante regresión.
Luego, para la segmentación, se aplicará una segmentación mediante métodos clásicos a partir de las bounding boxes preexistentes en el dataset, para obtener así una máscara con el segmento de interés para cada imagen.
Utilizando estas segmentaciones, se entrenará un modelo U-NET para obtener las máscaras deseadas. 
En todo momento, se hará uso de imágenes preprocesadas y rescaladas mediante reducción de ruido basado en análisis espectral y aumento sectorial del constraste. 
Finalmente, se evaluarán los resultados obtenidos mediante métricas de precisión.



## Base de datos
Para realizar nuestro trabajo, adquirimos las imágenes necesarias disponibles en [1]. 
Esta base de datos contiene 1528 imágenes de ultrasonido en vista sagital 2D, recolectadas del Shenzhen People's Hospital de 1519 gestantes.
Ademas, se cuenta con una tabla que indica una serie de bounding boxes para distintas estructuras identificables en cada una de las imágenes (entre ellas, la propia TN).
De las 1528, se tomaron solo aquellas identificadas como "Standard", y se filtraron los ejemplos en las que nos se contaba con una bounding box predefinida para la TN.
De esta manera, trabajaremos con 1100 imágenes de scan fetal abdominal y visión sagital del feto, en conjunto con sus correspondientes bounding boxes.

# Materiales y métodos

Para el desarrollo de este trabajo, se hará uso de los siguientes módulos de código abierto:
- numpy
- pandas
- scipy
- matplotlib
- openCV
- skimage
- tensorflow
- keras

Además, se desarrolló un módulo propio de denoising de imágenes, disponible en el archivo `denoising4GP.py`. Utilizando estas herramientas, se aplicaron las siguientes técnicas:
- Reescalado por interpolación
- Estimación del ruido mediante el espectro de frecuencia bidimensional
- Ecualización sectorizada del histograma mediante CLAHE
- Filtrado convolucional mediante kernels lineales
- Binarización por el método de Otsu
- Identificación de estructuras mediante el método de etiquetada
- Segmentación por region growing

Para cado uno de los casos presentado a continuación, el preprocesamiento aplicado consiste de los siguientes pasos:
1. Lectura de la imagen desde un archivo .png
2. Lectura de la bounding box correspondiente para la imagen
3. Reducción de ruido basada en el análisis espectral
4. Aumento de contraste mediante CLAHE 

Debido a que el dataset original no contaba con máscaras de segmentación semántica para el área anecogénica de la translucencia nucal, las cuales son de vital importancia para entrenar una red neuronal en esta tarea, se tuvo que desarrollar un algoritmo para segmentar dicha superficie a partir de la ROI de la bounding box. El pipeline adoptado consiste en:
1. Aplicar un filtro pasa altos para resaltar los bordes
2. Binarizar la imagen filtrada por Otsu
3. Etiquetar las áreas cerradas internas a la bounding box en la binarización
4. Seleccionar la etiqueta correspondiente al área más grande
5. Calcular el centroide de dicho segmento
6. Partiendo del centroide, aplicar el método de region growing sobre la imagen originalEstado del arte 

Finalmente, se entrenaron dos modelos de redes neuronales: un modelo basado en fine-tuning de VGG16 cuya salida es una regresión de las bounding boxes, y otro modelo basado en la U-NET cuya salida es una segmentación del área de la TN.

---